# Finding Feasible Counterfactual Explanations (FCX)

Feasible Counterfactual Explanations (FCX) is a novel framework that generates realistic and low-cost counterfactuals by enforcing both hard feasibility constraints provided by domain experts and soft causal constraints inferred from data. Built on a modified Variational Autoencoder and optimized with a multi-factor loss function, FCX produces sparse, diverse, and actionable counterfactuals while preserving causal relationships, offering both individual-level explanations and global model feasibility assessments across multiple datasets.

### Folktables Dataset Example

This notebook demonstrates preparation, training, and evaluation of the FCX models  
specifically for the **Folktables** dataset.  
We will:
1. Unpack the preprocessed data  
2. (Optional) Fine‑tune the black‑box model  
3. Train the unary and binary counterfactual generators  
4. Evaluate the trained generators  

In [ ]:
# Unzip the Adult dataset archive
#!7z x data.7z -o ./data
#!pip install torchvision==0.14.1
#!conda remove --force pytorch torchvision torchaudio
#!pip uninstall -y torch torchvision torchaudio

#!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117


Load paths

In [1]:
import sys, os
import importlib.util
exp_dir = os.path.abspath(os.path.join('..','..','humancompatible','explain'))
sys.path.insert(0, exp_dir)

fcx_dir = os.path.abspath(os.path.join('..','..','humancompatible','explain','fcx'))
sys.path.insert(0, fcx_dir)

# compute absolute path to the `scripts` folder
scripts_dir = os.path.abspath(
    os.path.join('..', '..', 'humancompatible', 'explain', 'fcx', 'scripts')
)
sys.path.append(scripts_dir)
# 1) Compute the full path to your script
script_path = os.path.abspath(
    os.path.join('..','..','humancompatible','explain','fcx','scripts','blackbox-model-train.py')
)
# 2) Create a module spec and module object
spec = importlib.util.spec_from_file_location("blackbox_model_train", script_path)
bb_mod = importlib.util.module_from_spec(spec)

# 3) Execute the module in its own namespace
spec.loader.exec_module(bb_mod)

# 4) Extract the function
train_blackbox = bb_mod.train_blackbox

## 1. (Optional) Fine‑tune the black‑box model for Folktables

Run this first if the provided checkpoint isn’t compatible.

In [2]:
train_blackbox('folktables_adult')

    age                                          workclass  \
0  18.0  Employee of a private for-profit company or bu...   
1  53.0                        Federal government employee   
2  41.0  Employee of a private for-profit company or bu...   
3  18.0  Self-employed in own not incorporated business...   
4  21.0                        Federal government employee   

                                         education  \
0                Some college but less than 1 year   
1                    GED or alternative credential   
2                      Regular high school diploma   
3                Some college but less than 1 year   
4  1 or more years of college credit but no degree   

                        marital_status  \
0  Never married or under 15 years old   
1  Never married or under 15 years old   
2  Never married or under 15 years old   
3  Never married or under 15 years old   
4  Never married or under 15 years old   

                                          occupat

KeyboardInterrupt: 

## 2. Train FCX on the Folktables dataset

Next, we train the **unary** generator, then the **binary** generator.

In [ ]:

from FCX_unary_generation_folktables_adult import train_unary_fcx_vae
#!pip install networkx==2.8
# Call it for the Adult dataset
train_unary_fcx_vae(
    'folktables_adult',
    base_data_dir='../../data/',
    base_model_dir='../models/',
    batch_size=2048, #2048
    epochs=25, #50
    validity=29.0,
    feasibility=192.0,
    margin=0.764
)

    age                                          workclass  \
0  18.0  Employee of a private for-profit company or bu...   
1  53.0                        Federal government employee   
2  41.0  Employee of a private for-profit company or bu...   
3  18.0  Self-employed in own not incorporated business...   
4  21.0                        Federal government employee   

                                         education  \
0                Some college but less than 1 year   
1                    GED or alternative credential   
2                      Regular high school diploma   
3                Some college but less than 1 year   
4  1 or more years of college credit but no degree   

                        marital_status  \
0  Never married or under 15 years old   
1  Never married or under 15 years old   
2  Never married or under 15 years old   
3  Never married or under 15 years old   
4  Never married or under 15 years old   

                                          occupat

## 3. Evaluate the trained models (Folktables)

Run evaluation scripts to compute validity and feasibility metrics.

In [ ]:

from evaluate_unary_folktables_adult import evaluate_folktables_adult
import warnings
warnings.filterwarnings("ignore")
res = evaluate_folktables_adult(
    base_data_dir='../../data/',
    base_model_dir='../models/',
    dataset_name='folktables_adult',
    pth_name = 'folktables_adult-margin-0.764-feasibility-192.0-validity-29.0-epoch-25-fcx-unary.pth'
)
print(res)

{10: 0, 9: 1, 5: 2, 0: 1, 2: 1, 11: 1, 4: 1, 1: 1, 8: 2, 6: 0, 3: 3, 7: 0}


ImportError: cannot import name 'evaluate_folktables_adult' from 'evaluate_unary_folktables_adult' (c:\Users\kleop\Downloads\explain-main\humancompatible\explain\fcx\evaluate_unary_folktables_adult.py)

Read the results from csv files